In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json

In [2]:
cache_path = 'mj_cache/'
output_file = 'output_512/dataset_mj.csv'

In [3]:
rfoa = pd.read_csv(cache_path + 'llc_rfo_accesses.csv')
rfom = pd.read_csv(cache_path + 'llc_rfo_misses.csv')
loada = pd.read_csv(cache_path + 'llc_load_accesses.csv')
loadm = pd.read_csv(cache_path + 'llc_load_misses.csv')
for i, j in zip(rfoa["id"], loada["id"]):
    if i != j: print(i)
for i, j in zip(rfom["id"], loadm["id"]):
    if i != j: print(i)
for i, j in zip(rfoa["id"], loadm["id"]):
    if i != j: print(i)

In [4]:
llc_accesses = pd.DataFrame()
llc_misses = pd.DataFrame()
llc_accesses["id"] = loadm["id"]
llc_misses["id"] = loadm["id"]
for i in ["1024", "2048", "4096", "8192"]:
    llc_accesses[i] = rfoa[i] + loada[i]
    llc_misses[i] = rfom[i] + loadm[i]

In [5]:
min_size = 512 * 1024

if os.path.exists(output_file):
    out_df = pd.read_csv(output_file)
else:
    out_df = pd.DataFrame()

for file in os.listdir('reuse_distances'):
    if len(out_df) > 0 and (file[:-4]+"_0")in set(out_df["id"]):
        print(file, " (found)")
        continue
    rd_df = pd.read_csv('reuse_distances/'+file)
    ctr1 = 0
    ctr2 = 0
    ctr4 = 0
    ctr8 = 0
    l1d_ctr1 = 0
    l1d_ctr2 = 0
    l1d_ctr4 = 0
    l1d_ctr8 = 0
    miss_ctr1 = 0
    miss_ctr2 = 0
    miss_ctr4 = 0
    miss_ctr8 = 0
    prev_ctr = 0
    
    # s = llc_accesses.index[llc_accesses['id'] == my_id["id"]].tolist()
    # s = s[0]
    for i in range(len(rd_df)):
        my_id = rd_df.iloc[i]
        row = llc_accesses[llc_accesses["id"] == my_id["id"]]
        # Accesses 
        ctr1 += int(row["1024"])
        ctr2 += int(row["2048"])
        ctr4 += int(row["4096"])
        ctr8 += int(row["8192"])
        
        # Misses
        row = llc_misses[llc_misses["id"] == my_id["id"]]
        miss_ctr1 += int(row["1024"])
        miss_ctr2 += int(row["2048"])
        miss_ctr4 += int(row["4096"])
        miss_ctr8 += int(row["8192"])
        if (ctr1 > min_size) or (i == len(rd_df) - 1 and prev_ctr == 0):
            out_line = rd_df.iloc[prev_ctr: i+1].sum(axis=0)
            out_line["id"] = file[:-4] + '_' + str(prev_ctr)
            out_line = pd.DataFrame(out_line).transpose()
            out_line["accesses_1024"] = ctr1
            out_line["misses_1024"] = miss_ctr1
            out_line["accesses_2048"] = ctr2
            out_line["misses_2048"] = miss_ctr2
            out_line["accesses_4096"] = ctr4
            out_line["misses_4096"] = miss_ctr4
            out_line["accesses_8192"] = ctr8
            out_line["misses_8192"] = miss_ctr8
            out_df = pd.concat([out_df, out_line], ignore_index = True)
            prev_ctr = i+1
            ctr1 = 0
            ctr2 = 0
            ctr4 = 0
            ctr8 = 0
            l1d_ctr1 = 0
            l1d_ctr2 = 0
            l1d_ctr4 = 0
            l1d_ctr8 = 0
            miss_ctr1 = 0
            miss_ctr2 = 0
            miss_ctr4 = 0
            miss_ctr8 = 0
    print(f"{file} : {out_df.shape}")
    out_df.to_csv(output_file,index=False)

400.perlbench-41B.csv  (found)
400.perlbench-50B.csv  (found)
401.bzip2-226B.csv  (found)
401.bzip2-277B.csv  (found)
401.bzip2-38B.csv  (found)
401.bzip2-7B.csv  (found)
403.gcc-16B.csv  (found)
403.gcc-17B.csv  (found)
403.gcc-48B.csv  (found)
410.bwaves-1963B.csv  (found)
410.bwaves-2097B.csv  (found)
410.bwaves-945B.csv  (found)
416.gamess-875B.csv  (found)
429.mcf-184B.csv  (found)
429.mcf-192B.csv  (found)
429.mcf-217B.csv  (found)
429.mcf-22B.csv  (found)
429.mcf-51B.csv  (found)
433.milc-127B.csv  (found)
433.milc-274B.csv  (found)
433.milc-337B.csv  (found)
434.zeusmp-10B.csv  (found)
435.gromacs-111B.csv  (found)
435.gromacs-134B.csv  (found)
435.gromacs-226B.csv  (found)
435.gromacs-228B.csv  (found)
436.cactusADM-1804B.csv  (found)
437.leslie3d-134B.csv  (found)
437.leslie3d-149B.csv  (found)
437.leslie3d-232B.csv  (found)
437.leslie3d-265B.csv  (found)
437.leslie3d-271B.csv  (found)
437.leslie3d-273B.csv  (found)
444.namd-120B.csv  (found)
444.namd-166B.csv  (found)
444.na

In [6]:
dct = {}
for i in out_df["id"]:
    if i in dct:
        print("Duplicate", i)
    dct[i] = 1

In [7]:
print(out_df)

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  50816880  17323312  11868112  7701840  3853504   
1     400.perlbench-41B_436  17361520   6023600   4203744  2591488  1828656   
2       400.perlbench-50B_0  58283520  17249296  14592592  9466576  7204896   
3          401.bzip2-226B_0   3098304   3037808   2698720  2692176  1999568   
4         401.bzip2-226B_54   1623056   1618144   1501200  1353728  1014736   
...                     ...       ...       ...       ...      ...      ...   
6177     657.xz_s-3167B_466   4869744   5968224   3301024  2821488  2218352   
6178     657.xz_s-3167B_574   4967104   7056944   3530208  2928336  2367040   
6179       657.xz_s-4994B_0  18063488  19199008   9437200  7269680  7833904   
6180     657.xz_s-4994B_343  13498912  14343616   6633152  5098416  6450640   
6181         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        6        7        8  ...  4160